In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')


def naive_nodewise_regression(Y_star, lambda_grid=None):
    """
    Implements Naive Nodewise Regression (Section 5.1.2).
    Uses GIC (Generalized Information Criterion) as in the paper.
    
    Parameters:
    -----------
    Y_star : np.ndarray, shape (n, p)
        Demeaned returns matrix (time x assets)
    lambda_grid : list or None
        Grid of lambda values to try. If None, creates default grid.
    
    Returns:
    --------
    Theta_hat : np.ndarray, shape (p, p)
        Estimated precision matrix
    """
    n, p = Y_star.shape

    # Initialize matrices
    Theta_hat = np.zeros((p, p))
    tau_squared = np.zeros(p)
    
    # Create lambda grid if not provided
    if lambda_grid is None:
        lambda_grid = np.logspace(-3, 1, 50)
    
    # For each asset j
    for j in range(p):
        # Step 1: Get y_j (target) and Y_{-j} (predictors)
        y_j = Y_star[:, j]
        Y_minus_j = np.delete(Y_star, j, axis=1)
        
        # Step 2-3: Estimate gamma_j using Lasso with GIC
        # GIC(λ) = log(σ²_λ) + |S_λ| * (log(p-1) / n) * log(log(n))
        best_gic = np.inf
        best_lambda = lambda_grid[0]
        best_gamma = None
        best_ssr = None
        
        for lam in lambda_grid:
            lasso = Lasso(alpha=2*lam, fit_intercept=False, max_iter=10000)
            lasso.fit(Y_minus_j, y_j)
            gamma_j = lasso.coef_
            
            # Compute SSR and number of non-zero coefficients
            residuals = y_j - Y_minus_j @ gamma_j
            ssr = np.sum(residuals ** 2)
            sigma_sq_lambda = ssr / n
            q_lambda = np.sum(np.abs(gamma_j) > 1e-8)
            
            # Compute GIC
            if sigma_sq_lambda > 1e-10: # Check for non-zero variance
                # GIC formula from paper
                gic = np.log(sigma_sq_lambda) + q_lambda * (np.log(p) / n) * np.log(np.log(n))
            else:
                gic = np.inf
            
            if gic < best_gic:
                best_gic = gic
                best_lambda = lam
                best_gamma = gamma_j.copy()
                best_ssr = ssr
        
        gamma_j_star = best_gamma
        

        tau_squared[j] = best_ssr / n + best_lambda * np.sum(np.abs(gamma_j_star))

        # [cite_start]Step 5: Form the j-th row of Theta_hat [cite: 579, 543-547]
        Theta_hat[j, j] = 1 / tau_squared[j]
        off_diag = -gamma_j_star / tau_squared[j]
        Theta_hat[j, :j] = off_diag[:j]
        Theta_hat[j, j+1:] = off_diag[j:]
    
    # Step 6: Symmetrize
    Theta_hat_sym = (Theta_hat + Theta_hat.T) / 2
    
    return Theta_hat_sym


def msr_weights(Theta_hat, mu):
    """
    Compute Maximum Sharpe Ratio portfolio weights.
    
    The maximum Sharpe ratio portfolio solves:
    max (w' mu) / sqrt(w' Sigma w)
    
    Solution (when mu represents excess returns):
    w ∝ Sigma^{-1} mu = Theta mu
    
    Then normalize so that sum(w) = 1.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected excess returns
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights (sum to 1)
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute unnormalized weights: w ∝ Theta mu
    w_unnorm = Theta_hat @ mu
    
    # Normalize to sum to 1
    weight_sum = np.sum(w_unnorm)
    
    if np.abs(weight_sum) < 1e-10:
        print('WARNING: Weight sum near zero, returning equal weights')
        return ones_p / p
    
    w_star = w_unnorm / weight_sum
    
    return w_star


def compute_portfolio_metrics(returns, weights):
    """
    Compute portfolio return, variance, and Sharpe ratio.
    """
    portfolio_returns = returns @ weights
    mean_return = np.mean(portfolio_returns)
    variance = np.var(portfolio_returns, ddof=1)
    sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
    
    return {
        'return': mean_return,
        'variance': variance,
        'sharpe_ratio': sharpe_ratio
    }


def backtest_nodewise_gmv(df, 
                          test_start_date='2000-01-31', 
                          test_end_date='2003-12-31',
                          lookback_window=180,
                          transaction_cost=0.005,
                          verbose=True):
    """
    Backtest Nodewise + MSR strategy with monthly rebalancing,
    180-month rolling window, and NaN filtering as per the paper.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ret_fwd_1
    test_start_date : str
        First date for out-of-sample returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for out-of-sample returns (format: 'YYYY-MM-DD')
    lookback_window : int
        Number of months in rolling training window (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.005 = 50 bps)
    verbose : bool
        If True, prints detailed log at each time step.
    
    Returns:
    --------
    results_df : pd.DataFrame
        DataFrame with columns: date, portfolio_return, cumulative_return
    metrics : dict
        Overall performance metrics
    """
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])
    
    # Get unique dates
    all_dates = sorted(df['datadate'].unique())
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback. Test start date {test_start_date} "
                         f"requires data back to {all_dates[test_start_idx - lookback_window]}, "
                         f"but only {test_start_idx} periods are available.")
    
    # Storage for results
    portfolio_returns = []
    portfolio_dates = []
    portfolio_weights_list = []
    portfolio_turnover_list = []
    portfolio_gross_returns = []
    
    # Track weights by permno (handles entry/exit)
    prev_weights_dict = {}
    prev_oos_returns_dict = {}
    prev_gross_return = 0.0
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("="*60)
        print("STARTING BACKTEST")
        print("="*60)
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        
        # Define the lookback window
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        # Get training data for this window
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date)]
        
        # Pivot to get returns matrix (time x assets)
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        
        # Reindex to ensure all dates are present
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates)
        
        # Filter assets with any NaNs in this window
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        n_train, p_current = Y.shape

        if verbose:
            print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                  f"Date: {current_date.strftime('%Y-%m-%d')}")
            print(f"  Window: {window_start_date.strftime('%Y-%m-%d')} to "
                  f"{window_end_date.strftime('%Y-%m-%d')}")
            print(f"  Assets: {p_current} with complete data")

        # Check for valid data
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  ⚠ Insufficient data (n={n_train}, p={p_current}), using prev weights")
            new_weights_dict = prev_weights_dict.copy()
        else:
            try:
                # Demean the returns
                Y_bar = Y.mean(axis=0)
                Y_star = Y - Y_bar
                
                if verbose:
                    print(f"  Running Nodewise Regression...")
                Theta_hat = naive_nodewise_regression(Y_star)

                # Compute expected returns (sample mean)
                mu = Y.mean(axis=0)
                
                if verbose:
                    print(f"  Computing MSR weights...")
                w_star = msr_weights(Theta_hat, mu)
                
                # Create weights dictionary
                new_weights_dict = {asset: w_star[i] for i, asset in enumerate(current_assets)}
                
            except Exception as e:
                if verbose:
                    print(f"  ✗ Error: {e}")
                    print(f"  Using previous weights")
                new_weights_dict = prev_weights_dict.copy()

        # Normalize weights to sum to 1
        weight_sum = sum(new_weights_dict.values())
        if weight_sum > 1e-10:
            new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
        else:
            if verbose:
                print("  ⚠ Zero weight sum, using previous weights")
            new_weights_dict = prev_weights_dict.copy()
        
        # --- 3. OOS Returns & Transaction Costs ---
        
        # Get out-of-sample returns for current month
        oos_data = df[df['datadate'] == current_date]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        
        # Filter out NaN returns
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # Find common assets between weights and returns
        common_assets = set(new_weights_dict.keys()) & set(oos_returns_dict.keys())
        
        if len(common_assets) == 0:
            if verbose:
                print("  ⚠ No common assets with valid returns, skipping period")
            continue
        
        # Filter to common assets and renormalize
        common_weights = {a: new_weights_dict[a] for a in common_assets}
        common_weight_sum = sum(common_weights.values())
        if common_weight_sum > 1e-10:
            common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
        else:
            if verbose:
                print("  ⚠ Zero weight sum after filtering, skipping period")
            continue
        
        # Compute gross portfolio return
        gross_return = sum(common_weights[a] * oos_returns_dict[a] for a in common_assets)
        
        # Sanity check
        if np.isnan(gross_return) or np.isinf(gross_return):
            if verbose:
                print(f"  ⚠ Invalid gross return: {gross_return}, skipping period")
            continue
        
        # Calculate transaction costs with proper weight adjustment
        if len(prev_weights_dict) > 0:
            # Step 1: Adjust ALL previous weights for returns
            adjusted_prev = {}
            
            for asset, prev_w in prev_weights_dict.items():
                if asset in prev_oos_returns_dict:
                    prev_r = prev_oos_returns_dict[asset]
                    if abs(1 + prev_gross_return) > 1e-6:
                        adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
                else:
                    print("SUSPICIOUS")
                    # Asset had weight but no return data
                    if abs(1 + prev_gross_return) > 1e-6:
                        adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
            
            # Step 2: Calculate turnover
            all_assets = set(adjusted_prev.keys()) | set(common_weights.keys())
            
            turnover = 0.0
            for asset in all_assets:
                old_w = adjusted_prev.get(asset, 0.0)
                new_w = common_weights.get(asset, 0.0)
                turnover += abs(new_w - old_w)
            
            # Transaction cost on end-of-period portfolio value
            tc = transaction_cost * (1 + gross_return) * turnover
        
        else:
            # First period: no previous positions, buying into everything
            turnover = sum(abs(w) for w in common_weights.values())
            tc = transaction_cost * (1 + gross_return) * turnover
        
        # Net return
        net_return = gross_return - tc
        
        # Store results
        portfolio_returns.append(net_return)
        portfolio_dates.append(current_date)
        portfolio_weights_list.append(common_weights.copy())
        portfolio_turnover_list.append(turnover)
        portfolio_gross_returns.append(gross_return)
        
        # Update previous values for next iteration
        prev_weights_dict = common_weights.copy()
        prev_oos_returns_dict = {a: oos_returns_dict[a] for a in common_assets}
        prev_gross_return = gross_return
        
        if verbose:
            print(f"  Gross: {gross_return:>8.5f} | Turnover: {turnover:>6.4f} | "
                  f"TC: {tc:>8.6f} | Net: {net_return:>8.5f}")

    if verbose:
        print("\n" + "="*60)
        print("BACKTEST COMPLETE")
        print("="*60)
    
    # --- 4. Compile Results ---
    results_df = pd.DataFrame({
        'date': portfolio_dates,
        'portfolio_return': portfolio_returns,
        'portfolio_gross_return': portfolio_gross_returns,
        'portfolio_weights': portfolio_weights_list,
        'portfolio_turnover': portfolio_turnover_list
    })
    results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
    
    # Compute overall metrics
    if len(portfolio_returns) > 0:
        mean_return = np.mean(portfolio_returns)
        variance = np.var(portfolio_returns, ddof=1)
        sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
        
        # Annualized metrics (monthly data)
        annual_return = mean_return * 12
        annual_volatility = np.sqrt(variance * 12)
        annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
        
        metrics = {
            'mean_return': mean_return,
            'variance': variance,
            'sharpe_ratio': sharpe_ratio,
            'annual_return': annual_return,
            'annual_volatility': annual_volatility,
            'annual_sharpe_ratio': annual_sharpe,
            'total_return': results_df['cumulative_return'].iloc[-1],
            'avg_turnover': np.mean(portfolio_turnover_list),
            'n_periods': len(portfolio_returns)
        }
    else:
        metrics = {
            'mean_return': 0,
            'variance': 0,
            'sharpe_ratio': 0,
            'annual_return': 0,
            'annual_volatility': 0,
            'annual_sharpe_ratio': 0,
            'total_return': 0,
            'avg_turnover': 0,
            'n_periods': 0
        }
    
    return results_df, metrics

In [2]:
# Assuming you have loaded your dataframe as 'df'
df = pd.read_csv('../../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = (df.groupby('permno')['ret_excess'].shift(-1) )

In [3]:
results_df, metrics = backtest_nodewise_gmv(
    df,
    test_start_date='2020-01-31',  # Last date of training period
    test_end_date='2024-04-30',   # Last date of testing period
    lookback_window=180,
    transaction_cost=0.001
)
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics['annual_sharpe_ratio']:.4f}")

STARTING BACKTEST

[1/52] Date: 2020-01-31
  Window: 2005-01-31 to 2019-12-31
  Assets: 252 with complete data
  Running Nodewise Regression...
  Computing MSR weights...
  Gross: -0.08808 | Turnover: 1.1269 | TC: 0.001028 | Net: -0.08911

[2/52] Date: 2020-02-29
  Window: 2005-02-28 to 2020-01-31
  Assets: 252 with complete data
  Running Nodewise Regression...
  Computing MSR weights...
  Gross: -0.12050 | Turnover: 0.0571 | TC: 0.000050 | Net: -0.12055

[3/52] Date: 2020-03-31
  Window: 2005-03-31 to 2020-02-29
  Assets: 253 with complete data
  Running Nodewise Regression...
  Computing MSR weights...
  Gross:  0.09909 | Turnover: 0.1242 | TC: 0.000136 | Net:  0.09896

[4/52] Date: 2020-04-30
  Window: 2005-04-30 to 2020-03-31
  Assets: 251 with complete data
  Running Nodewise Regression...
  Computing MSR weights...
  Gross:  0.04097 | Turnover: 0.0868 | TC: 0.000090 | Net:  0.04088

[5/52] Date: 2020-05-31
  Window: 2005-05-31 to 2020-04-30
  Assets: 250 with complete data
  Run

In [28]:

with pd.option_context("display.max_rows", None):
    print(results_df['portfolio_weights'][0])

{34817: 0.00242249395553771, 20482: 0.006805074384082329, 49154: 0.003278833820521948, 52230: 0.0026929685513930827, 71175: 0.0036082967359080953, 48653: 0.0014864570824605881, 60943: 0.003418292246796153, 49680: 0.008441086495762135, 34833: 0.0037389081747815998, 59408: -0.003849968835480621, 87055: 0.006589806519631617, 72726: 0.002504080197924304, 60442: 0.005089846212758344, 23579: -0.006912055774039984, 13856: 0.010727714153810112, 43553: 0.004285533231924262, 76841: 0.0017716945499136768, 11308: 0.009798519000077924, 24109: 0.008377224229564613, 19502: 0.0029733566295818865, 66093: 0.007230487455301651, 57904: 0.0038917573025133373, 75825: 0.002124582489416908, 75828: 0.0015300394675521476, 86580: -0.0013756608637485795, 44601: 0.0036601648027149783, 60986: 0.0016762223132872065, 28222: 0.006279107597662725, 22592: 0.006825964457490043, 39490: 0.000922192264267681, 24643: 0.0007367122289558844, 59459: 0.00700499859404927, 21573: 0.0014031483106038628, 17478: 0.0037473504861341086

In [29]:
metrics['total_return']

0.3756070714683275

In [30]:
metrics['variance']

0.0023410463166250403

In [31]:
results_df['portfolio_turnover'].mean()

0.09082020565331547